
# 01 — Load OSM data (Geofabrik) and build a first map

This notebook loads OpenStreetMap shapefiles from the Geofabrik download for a Russian federal district, filters a single region (default: **Vladimir Oblast**), extracts **MFC** (public service) and **telecom towers**, and produces:
- a quick static plot,
- an **interactive Folium map**,
- and saves outputs to `reports/figures/`.

> Replace `TARGET_REGION_SUBSTR` below if you want a different region (e.g., `"Moscow"`, `"Tula"`, `"Nizhny"`).


In [ ]:

# If running in Google Colab, uncomment the next line to install deps:
# !pip install geopandas folium osmnx shapely matplotlib

import os
from pathlib import Path
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point
import folium

# --- Repo-relative paths ---
REPO_ROOT = Path(".").resolve()
DATA_RAW = REPO_ROOT / "data" / "raw"
DATA_PROCESSED = REPO_ROOT / "data" / "processed"
FIG_DIR = REPO_ROOT / "reports" / "figures"

# Ensure output dirs exist
DATA_PROCESSED.mkdir(parents=True, exist_ok=True)
FIG_DIR.mkdir(parents=True, exist_ok=True)

# Filenames from Geofabrik (Central Federal District shp.zip after extraction)
ADMIN_SHP = DATA_RAW / "gis_osm_admin_a_free_1.shp"
POIS_SHP  = DATA_RAW / "gis_osm_pois_free_1.shp"

print("Repo root:", REPO_ROOT)
print("Admin file exists:", ADMIN_SHP.exists())
print("POIs  file exists:", POIS_SHP.exists())


In [ ]:

# --- Load shapefiles ---
admin = gpd.read_file(ADMIN_SHP)
pois  = gpd.read_file(POIS_SHP)

print("Admin rows:", len(admin))
print("POIs  rows:", len(pois))
print("Admin columns:", list(admin.columns))
print("POIs  columns:", list(pois.columns))

# Preview
display(admin.head(2))
display(pois.head(2))


In [ ]:

# --- Pick target region by substring in the 'name' column ---
TARGET_REGION_SUBSTR = "Vladimir"  # change to e.g. "Moscow", "Tula", "Nizhny"

region = admin[admin["name"].str.contains(TARGET_REGION_SUBSTR, case=False, na=False)].copy()
assert len(region) > 0, f"No admin polygons matched substring: {TARGET_REGION_SUBSTR!r}"

# Keep only polygons (drop points/lines if any slipped in)
region = region[region.geometry.notna()].copy()

print("Matched polygons:", len(region))
region.boundary.plot(figsize=(7,6))
plt.title(f"Region filter: contains '{TARGET_REGION_SUBSTR}'")
plt.show()


In [ ]:

# --- Spatial filter POIs to the selected region (within) ---
# Reproject to a common CRS if needed
if pois.crs != region.crs:
    pois = pois.to_crs(region.crs)

# Spatial join: POIs inside the region
pois_in_region = gpd.sjoin(pois, region, how="inner", predicate="within")

# Select MFC (public services) and telecom towers
mfc = pois_in_region[pois_in_region.get("fclass") == "public_service"].copy()

# Some datasets use different tagging for towers; include a few likely classes
tower_like = {"tower", "communication", "telecom", "mast"}
towers = pois_in_region[pois_in_region.get("fclass").isin(tower_like)].copy()

print("POIs inside region:", len(pois_in_region))
print("MFC count:", len(mfc))
print("Towers-like count:", len(towers))

# Quick static map
ax = region.plot(figsize=(7,6))
if len(mfc):
    mfc.plot(ax=ax, markersize=5)
if len(towers):
    towers.plot(ax=ax, markersize=3)
plt.title("Region + MFC + Towers (quicklook)")
plt.tight_layout()
plt.savefig(FIG_DIR / "quicklook_static.png", dpi=200)
plt.show()


In [ ]:

# --- Save processed layers for reuse ---
region_out = DATA_PROCESSED / "region.geojson"
mfc_out = DATA_PROCESSED / "mfc.geojson"
towers_out = DATA_PROCESSED / "towers.geojson"

region.to_file(region_out, driver="GeoJSON")
if len(mfc):
    mfc.to_file(mfc_out, driver="GeoJSON")
if len(towers):
    towers.to_file(towers_out, driver="GeoJSON")

print("Saved:", region_out)
if len(mfc): print("Saved:", mfc_out)
if len(towers): print("Saved:", towers_out)


In [ ]:

# --- Build interactive Folium map ---
# Set a rough initial map center (region centroid)
centroid = region.to_crs(epsg=4326).geometry.unary_union.centroid
m = folium.Map(location=[centroid.y, centroid.x], zoom_start=7)

# Add region boundary
folium.GeoJson(region.to_crs(epsg=4326).to_json(), name="Region").add_to(m)

# Add MFC layer
if len(mfc):
    folium.GeoJson(
        mfc.to_crs(epsg=4326).to_json(),
        name=f"MFC ({len(mfc)})",
        marker=None,
        tooltip=folium.features.GeoJsonTooltip(fields=["name"], aliases=["Name"])
    ).add_to(m)

# Add Towers layer
if len(towers):
    folium.GeoJson(
        towers.to_crs(epsg=4326).to_json(),
        name=f"Towers ({len(towers)})",
        marker=None,
        tooltip=folium.features.GeoJsonTooltip(fields=["name"], aliases=["Name"])
    ).add_to(m)

folium.LayerControl().add_to(m)

html_path = FIG_DIR / "region_map.html"
m.save(html_path)
html_path
